In [13]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE >= '2021-01-01'
    AND SERVICE_DATE < '2025-01-01'
),
WEIGHT_DX AS (
    SELECT DISTINCT MEMBER_ID
    FROM Actuarial_AH.DBO.SN_Medical
    WHERE ICD10_SECTION LIKE '%Overweight%'
)

SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
       YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
       YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
       M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
       M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN Actuarial_AH.dbo.SN_Member MB ON MB.MEMBER_ID = M.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
)
AND MB.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHT_DX
)
"""
med = pd.read_sql(med, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_26976\1900730163.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  med = pd.read_sql(med, conn)


In [4]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE BETWEEN '2021-01-01' AND '2024-12-31'
),
WEIGHT_DX AS (
    SELECT DISTINCT MEMBER_ID
    FROM Actuarial_AH.DBO.SN_Medical
    WHERE ICD10_SECTION LIKE '%Overweight%'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE, 
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
JOIN Actuarial_AH.dbo.SN_Medical MD ON MD.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND MD.ICD10_SECTION LIKE '%Overweight%'
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_26976\2795320354.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [5]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
WHERE ART_DRUGGRPS_L2 IN ('GLP-1 Agonists', 'Weight Loss Agents')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
),
WEIGHT_DX AS (
    SELECT DISTINCT MEMBER_ID
    FROM Actuarial_AH.DBO.SN_Medical
    WHERE ICD10_SECTION LIKE '%Overweight%'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
JOIN Actuarial_AH.dbo.SN_Medical MD ON M.MEMBER_ID = MD.MEMBER_ID
WHERE M.MEMBER_ID NOT IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MD.ICD10_SECTION LIKE '%Overweight%'

"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_26976\4259235446.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [6]:
member

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,mma-c1d477e010df4d1ab387a9bc4d01d6fd,0.0,CT,Subscriber,Male,34,Millenials
1,mma-036ffc20ff794595a44950a4729f66d3,0.0,LA,Subscriber,Female,62,Baby Boomers
2,mma-036ffc20ff794595a44950a4729f66d3,0.0,LA,Subscriber,Female,62,Baby Boomers
3,mma-4c45de73f32445699b80d8d493793dff,0.0,MA,Subscriber,Male,57,Generation X
4,mma-4c45de73f32445699b80d8d493793dff,0.0,MA,Subscriber,Male,57,Generation X
...,...,...,...,...,...,...,...
154559,mma-565511f8a56541379d9fd2f16e419df3,0.0,IL,Subscriber,Female,64,Baby Boomers
154560,mma-565511f8a56541379d9fd2f16e419df3,0.0,IL,Subscriber,Female,64,Baby Boomers
154561,mma-cb4e69f3bcba4d6fa609f5e4be267b0a,0.0,NC,Spouse,Female,42,Millenials
154562,mma-e05f6060ee6c448ebee94236610c8ebb,0.0,KY,Subscriber,Male,45,Generation X


In [7]:
member.to_csv('no-glp-1-member-claims.csv', index=False)
med.to_csv('med-no-glp-claims.csv', index=False)
rx.to_csv('rx-no-glp-claims.csv', index=False)

In [65]:
med = pd.read_csv('med-no-glp-claims.csv')
rx = pd.read_csv('rx-no-glp-claims.csv')
member = pd.read_csv('no-glp-1-member-claims.csv')

In [66]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,PROCEDURE_DESC,ARTTOS_V2_L1,ARTTOS_V2_L3,DX_IS_CHRONIC,ICD10_CHAPTER,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1915160.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-21,2020,0.34,0.00,0.00,Hypertensive chronic kidney disease with stage...,Measurement of inorganic phosphorus (phosphate...,Outpatient,Pathology/Lab (O15),1.00,Diseases of the circulatory system,Hypertensive diseases,Hypertensive chronic kidney disease,Renal Failure,0,0,0,0,0,0,0
1,2292193.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-15,2020,1.39,0.00,0.00,"Chronic kidney disease, stage 4 (severe)",Urine osmolality (concentration) measurement (...,Outpatient,Pathology/Lab (O15),1.00,Diseases of the genitourinary system,Acute kidney failure and chronic kidney disease,Chronic kidney disease (CKD),Anomaly: Defects of Kidney,0,0,0,0,0,0,0
2,1025207.00,mma-f41c6eb76982421f9126a357a7e5f100,2021-01-01,2021,2020-08-11,2020,27.28,0.00,0.00,Prediabetes,"Blood count; complete (CBC), automated (Hgb, H...",Outpatient,Pathology/Lab (O15),0.00,"Symptoms, signs and abnormal clinical and labo...","Abnormal findings on examination of blood, wit...",Elevated blood glucose level,Diabetes Mellitus Type 2 & Unspec Type Mainten...,0,0,0,0,0,0,0
3,1915160.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-21,2020,0.34,0.00,0.00,Hypertensive chronic kidney disease with stage...,Syphilis detection test (CPT 86592),Outpatient,Pathology/Lab (O15),1.00,Diseases of the circulatory system,Hypertensive diseases,Hypertensive chronic kidney disease,Renal Failure,0,0,0,0,0,0,0
4,2673633.00,mma-b66382599c824f5e99482fec1a65c0be,2021-01-01,2021,2020-12-21,2020,4.00,0.00,0.00,Nonrheumatic tricuspid (valve) insufficiency,"Echocardiography, transthoracic, real-time wit...",Professional,Cardiovascular (P54),1.00,Diseases of the circulatory system,Other forms of heart disease,Nonrheumatic tricuspid valve disorders,Renal Failure,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880118,1280854.00,mma-2b9ce5d044844a6ab9b7b21911f7609d,2024-12-31,2024,2024-12-18,2024,190.85,0.00,0.00,"Unspecified lump in the left breast, upper inn...",Mri Breast C-+ W/Cad Bi (CPT 77049),Professional,CT/MRI/PET - MRI (P57b),0.00,Diseases of the genitourinary system,Disorders of breast,Unspecified lump in breast,Other Breast Disorders,0,0,0,0,0,0,0
3880119,3404594.00,mma-63a4609b2c394a00b2932195be7adddc,2024-12-31,2024,2024-11-26,2024,749.86,0.00,0.00,"Disorder of adrenal gland, unspecified","CT of abdomen without contrast, followed by co...",Professional,CT/MRI/PET - CT Scan (P59a),1.00,"Endocrine, nutritional and metabolic diseases",Disorders of other endocrine glands,Other disorders of adrenal gland,Adrenal Insufficiency,0,0,0,0,0,0,0
3880120,819006.00,mma-af5ce9ec6fdd46a0ad1fdffb2e2c543a,2024-12-31,2024,2024-12-18,2024,5.25,0.00,0.00,Encounter for screening for diabetes mellitus,Insertion of needle into vein for collection o...,Professional,Office - Venipuncture (P63b),0.00,Factors influencing health status and contact ...,Persons encountering health services for exami...,Encounter for screening for other diseases and...,Encounter for Preventive Health Services,0,0,0,0,0,0,0
3880121,4732981.00,mma-62edd7133abd47c98a82f0b2beae0c08,2024-12-31,2024,2024-11-25,2024,3.55,0.00,0.00,Other artificial openings of urinary tract status,"Ostomy skin barrier, powder, per oz (HCPCS A4371)",Other,DME and Supplies (P84),1.00,Factors influencing health status and contact ...,Persons with potential health hazards related ...,Artificial opening status,Sleep Disorders,0,0,0,0,0,0,0


In [67]:
rx_wla = rx[(rx['ART_DRUGGRPS_L2'] == 'Weight Loss Agents') | (rx['ART_DRUGGRPS_L1'] == 'GLP-1 Agonists')]
rx_wla.DRUG_NAME_PREFERRED.unique()

array(['Saxenda'], dtype=object)

In [68]:
member = member.drop_duplicates()
member.reset_index(inplace=True)


In [69]:
med.MEMBER_ID.nunique()

19967

In [70]:
rx.MEMBER_ID.nunique()

19556

In [76]:
primary_chapter_removed = med[med['ICD10_CHAPTER'] == 'Pregnancy, childbirth and the puerperium']
primary_section_removed = med[med['ICD10_SECTION'] == 'Malignant neoplasm of thyroid gland']
primary_category_removed = med[(med['ICD10_CATEGORY'] == 'Acute pancreatitis') | (med['ICD10_CATEGORY'] == 'Acute pancreatitis')]

In [79]:
# Filter to find members with the specified conditions
members_to_remove = med[(med['ICD10_CHAPTER'] == 'Pregnancy, childbirth and the puerperium') | (med['ICD10_SECTION'] == 'Malignant neoplasm of thyroid gland') |
(med['ICD10_CATEGORY'] == 'Acute pancreatitis') | (med['ICD10_CATEGORY'] == '')]['MEMBER_ID'].unique()

# Remove those members from the original DataFrame
rx_members_remove = rx[~rx['MEMBER_ID'].isin(members_to_remove)]
med_members_remove = med[~med['MEMBER_ID'].isin(members_to_remove)]
members_remove = member[~member['MEMBER_ID'].isin(members_to_remove)]

In [80]:
med_members_remove.MEMBER_ID.nunique()

18834

In [81]:
rx_members_remove.MEMBER_ID.nunique()

18433

In [82]:
med_agg = med_members_remove.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)

In [83]:
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,29731.95,1338.85,4440.36,21202.37
1,mma-00024d2d45224e76915920b058364ff1,180.70,269.09,199.06,680.49
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,5402.26,14290.44,8734.14,5885.90
3,mma-0007e20948f4487d97f4eb2981613ea1,1030.05,844.93,0.00,2181.59
4,mma-0008856832474235be7f58214e0cabc3,489.96,156.80,269.88,197.48
...,...,...,...,...,...
18829,mma-ffe711349c65480c9740ba217d4ad2cc,1376.62,2364.41,2065.03,668.59
18830,mma-ffe7972f15694551af377f4666f57f60,593.13,1291.41,441.64,0.00
18831,mma-ffe9040f10b9429eb7ce09ceaaf1e158,483.38,158.63,493.09,489.41
18832,mma-fff2da3c741f4879991e6f65a74c58e9,440.81,1891.86,2379.53,13742.94


In [84]:
# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48
...,...,...,...,...,...
18829,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59
18830,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00
18831,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41
18832,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94


In [85]:
med_agg.fillna(0, inplace=True)
med_agg.isnull().sum()

MEMBER_ID    0
Med_2021     0
Med_2022     0
Med_2023     0
Med_2024     0
dtype: int64

In [86]:
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48
...,...,...,...,...,...
18829,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59
18830,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00
18831,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41
18832,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94


In [87]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37,-13732.67,-39.31
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49,468.17,220.50
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90,-461.76,-7.27
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59,971.28,80.25
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48,-378.22,-65.70
...,...,...,...,...,...,...,...
18829,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59,-948.94,-58.67
18830,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00,-696.93,-100.00
18831,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41,-78.56,-13.83
18832,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94,13224.99,2553.32


In [88]:
med_agg.fillna(0, inplace=True)
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
dtype: int64

In [89]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,34935.04,1495.50,4746.74,21202.37,-13732.67,-39.31,-3794.68
1,mma-00024d2d45224e76915920b058364ff1,212.32,300.57,212.80,680.49,468.17,220.50,131.67
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,6347.66,15962.42,9336.80,5885.90,-461.76,-7.27,-801.09
3,mma-0007e20948f4487d97f4eb2981613ea1,1210.31,943.79,0.00,2181.59,971.28,80.25,197.01
4,mma-0008856832474235be7f58214e0cabc3,575.70,175.15,288.50,197.48,-378.22,-65.70,-102.13
...,...,...,...,...,...,...,...,...
18829,mma-ffe711349c65480c9740ba217d4ad2cc,1617.53,2641.05,2207.52,668.59,-948.94,-58.67,-328.03
18830,mma-ffe7972f15694551af377f4666f57f60,696.93,1442.50,472.11,0.00,-696.93,-100.00,-306.12
18831,mma-ffe9040f10b9429eb7ce09ceaaf1e158,567.97,177.19,527.11,489.41,-78.56,-13.83,11.42
18832,mma-fff2da3c741f4879991e6f65a74c58e9,517.95,2113.21,2543.72,13742.94,13224.99,2553.32,4010.55


In [90]:
rx_agg = rx_members_remove.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,1248.02,417.37,369.43,671.02
1,mma-00024d2d45224e76915920b058364ff1,42.40,0.00,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,1968.26,1783.30,10715.61
3,mma-0007e20948f4487d97f4eb2981613ea1,417.20,143.25,43.80,189.95
4,mma-0008856832474235be7f58214e0cabc3,62.00,0.00,0.00,82.36
...,...,...,...,...,...
18428,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,NaN,172.44,768.54
18429,mma-ffe7972f15694551af377f4666f57f60,0.00,2194.32,1220.24,0.00
18430,mma-ffe9040f10b9429eb7ce09ceaaf1e158,677.35,943.25,762.00,546.50
18431,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,97.96,531.72,578.40


In [91]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02
1,mma-00024d2d45224e76915920b058364ff1,55.54,0.00,NaN,NaN
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,2361.91,1966.98,10715.61
3,mma-0007e20948f4487d97f4eb2981613ea1,546.53,171.90,48.31,189.95
4,mma-0008856832474235be7f58214e0cabc3,81.22,0.00,0.00,82.36
...,...,...,...,...,...
18428,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,NaN,190.20,768.54
18429,mma-ffe7972f15694551af377f4666f57f60,0.00,2633.18,1345.92,0.00
18430,mma-ffe9040f10b9429eb7ce09ceaaf1e158,887.33,1131.90,840.49,546.50
18431,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,117.55,586.49,578.40


In [92]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [93]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02,-963.89,-58.96,-298.50
1,mma-00024d2d45224e76915920b058364ff1,55.54,0.00,0.00,0.00,-55.54,-100.00,-16.66
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,2361.91,1966.98,10715.61,10715.61,inf,3175.19
3,mma-0007e20948f4487d97f4eb2981613ea1,546.53,171.90,48.31,189.95,-356.58,-65.24,-119.33
4,mma-0008856832474235be7f58214e0cabc3,81.22,0.00,0.00,82.36,1.14,1.40,0.34
...,...,...,...,...,...,...,...,...
18428,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,0.00,190.20,768.54,768.54,inf,249.58
18429,mma-ffe7972f15694551af377f4666f57f60,0.00,2633.18,1345.92,0.00,0.00,NaN,-128.73
18430,mma-ffe9040f10b9429eb7ce09ceaaf1e158,887.33,1131.90,840.49,546.50,-340.83,-38.41,-131.39
18431,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,117.55,586.49,578.40,578.40,inf,220.41


In [94]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [95]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0001ed26305b48f99696f11dfcea17b7,1634.91,500.84,407.48,671.02,-963.89,-58.96,-298.50
1,mma-00024d2d45224e76915920b058364ff1,55.54,0.00,0.00,0.00,-55.54,-100.00,-16.66
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,0.00,2361.91,1966.98,10715.61,10715.61,0.00,3175.19
3,mma-0007e20948f4487d97f4eb2981613ea1,546.53,171.90,48.31,189.95,-356.58,-65.24,-119.33
4,mma-0008856832474235be7f58214e0cabc3,81.22,0.00,0.00,82.36,1.14,1.40,0.34
...,...,...,...,...,...,...,...,...
18428,mma-ffe711349c65480c9740ba217d4ad2cc,0.00,0.00,190.20,768.54,768.54,0.00,249.58
18429,mma-ffe7972f15694551af377f4666f57f60,0.00,2633.18,1345.92,0.00,0.00,NaN,-128.73
18430,mma-ffe9040f10b9429eb7ce09ceaaf1e158,887.33,1131.90,840.49,546.50,-340.83,-38.41,-131.39
18431,mma-fff2da3c741f4879991e6f65a74c58e9,0.00,117.55,586.49,578.40,578.40,0.00,220.41


In [97]:
member_mrg = members_remove.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg       

,index,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,1,mma-036ffc20ff794595a44950a4729f66d3,0.00,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,345.11,11264.21,14074.46,1549.84,1204.73,349.09,642.44
1,3,mma-4c45de73f32445699b80d8d493793dff,0.00,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,939.01,982.04,2713.93,2531.62,1592.61,169.61,650.97
2,9,mma-f8d8a13411e045629b92e106d35b3cd8,0.00,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,522.14,0.00,0.00,48.51,-473.63,-90.71,-142.09
3,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,2.00,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,1929.71,0.00,0.00,0.00,-1929.71,-100.00,-578.91
4,19,mma-ed99a4f01306482caec74ec3d315501c,0.00,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,1392.91,233.62,1520.28,1520.28,0.00,340.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18406,154531,mma-38955c494c7c4c888d0f65459d8959e3,0.00,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,1603.44,2200.50,1308.08,2179.17,575.73,35.91,83.48
18407,154540,mma-46d15af9af544cd6b09133c45ec04642,0.00,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,366.80,1509.65,930.84,22.05,-344.75,-93.99,-161.31
18408,154547,mma-c0b85487143b49dcbd14a6c3794ba398,0.00,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,550.18,846.40,846.40,0.00,308.94
18409,154552,mma-565511f8a56541379d9fd2f16e419df3,0.00,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,2200.72,2374.92,1961.08,630.36,-1570.36,-71.36,-512.49


In [98]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS'])
member1

,index,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,1,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,345.11,11264.21,14074.46,1549.84,1204.73,349.09,642.44
1,3,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,939.01,982.04,2713.93,2531.62,1592.61,169.61,650.97
2,9,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,522.14,0.00,0.00,48.51,-473.63,-90.71,-142.09
3,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,1929.71,0.00,0.00,0.00,-1929.71,-100.00,-578.91
4,19,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,1392.91,233.62,1520.28,1520.28,0.00,340.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18406,154531,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,1603.44,2200.50,1308.08,2179.17,575.73,35.91,83.48
18407,154540,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,366.80,1509.65,930.84,22.05,-344.75,-93.99,-161.31
18408,154547,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,550.18,846.40,846.40,0.00,308.94
18409,154552,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,2200.72,2374.92,1961.08,630.36,-1570.36,-71.36,-512.49


In [99]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-0001ed26305b48f99696f11dfcea17b7,1.00,0,6,1,2,19
1,mma-00024d2d45224e76915920b058364ff1,1.00,0,0,0,7,0
2,mma-0006f22cc2ed4e6da73eb73991bbbf57,1.00,0,22,0,4,0
3,mma-0007e20948f4487d97f4eb2981613ea1,1.00,0,18,17,0,0
4,mma-0008856832474235be7f58214e0cabc3,1.00,0,4,4,4,0
...,...,...,...,...,...,...,...
19962,mma-ffe7972f15694551af377f4666f57f60,1.00,0,1,4,2,0
19963,mma-ffe9040f10b9429eb7ce09ceaaf1e158,1.00,0,8,0,2,0
19964,mma-fff2da3c741f4879991e6f65a74c58e9,1.00,0,5,2,4,18
19965,mma-fffb3143f0b84fa690e26308813fceb0,1.00,0,12,1,3,23


In [100]:
merge_member = member1.merge(med_util, on='MEMBER_ID')
merge_member

,index,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,345.11,11264.21,14074.46,1549.84,1204.73,349.09,642.44,1.00,0,10,1,6,0
1,3,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,939.01,982.04,2713.93,2531.62,1592.61,169.61,650.97,1.00,13,4,2,2,0
2,9,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,522.14,0.00,0.00,48.51,-473.63,-90.71,-142.09,1.00,0,6,1,4,0
3,17,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,1929.71,0.00,0.00,0.00,-1929.71,-100.00,-578.91,1.00,0,4,0,1,0
4,19,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,1392.91,233.62,1520.28,1520.28,0.00,340.15,1.00,0,7,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18406,154531,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,1603.44,2200.50,1308.08,2179.17,575.73,35.91,83.48,1.00,0,4,0,5,0
18407,154540,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,366.80,1509.65,930.84,22.05,-344.75,-93.99,-161.31,1.00,10,0,0,4,0
18408,154547,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,550.18,846.40,846.40,0.00,308.94,1.00,6,12,0,3,0
18409,154552,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,2200.72,2374.92,1961.08,630.36,-1570.36,-71.36,-512.49,1.00,16,13,0,0,0


In [101]:
merge_member.drop(columns='index', inplace=True)

In [102]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)
merge_member

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,345.11,11264.21,14074.46,1549.84,1204.73,349.09,642.44,1.00,0,10,1,6,0
1,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,939.01,982.04,2713.93,2531.62,1592.61,169.61,650.97,1.00,13,4,2,2,0
2,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,522.14,0.00,0.00,48.51,-473.63,-90.71,-142.09,1.00,0,6,1,4,0
3,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,1929.71,0.00,0.00,0.00,-1929.71,-100.00,-578.91,1.00,0,4,0,1,0
4,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,1392.91,233.62,1520.28,1520.28,0.00,340.15,1.00,0,7,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18406,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,1603.44,2200.50,1308.08,2179.17,575.73,35.91,83.48,1.00,0,4,0,5,0
18407,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,366.80,1509.65,930.84,22.05,-344.75,-93.99,-161.31,1.00,10,0,0,4,0
18408,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,550.18,846.40,846.40,0.00,308.94,1.00,6,12,0,3,0
18409,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,2200.72,2374.92,1961.08,630.36,-1570.36,-71.36,-512.49,1.00,16,13,0,0,0


In [103]:
episodes = med.ICD10_CHAPTER.value_counts()
episodes.head(10)

ICD10_CHAPTER
Factors influencing health status and contact with health services                         788093
Diseases of the musculoskeletal system and connective tissue                               600956
Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified    442493
Endocrine, nutritional and metabolic diseases                                              339513
Mental, Behavioral and Neurodevelopmental disorders                                        210955
Diseases of the circulatory system                                                         202392
Diseases of the genitourinary system                                                       202022
Diseases of the nervous system                                                             167391
Diseases of the digestive system                                                           163603
Diseases of the respiratory system                                                         140852
Name: 

In [104]:
episodes_cost = med.groupby('ICD10_CHAPTER')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,ICD10_CHAPTER,MEDICAL_PAID_AMOUNT
10,Diseases of the musculoskeletal system and con...,76349045.93
16,Factors influencing health status and contact ...,74186255.83
5,Diseases of the circulatory system,46214797.96
6,Diseases of the digestive system,45361834.75
19,Neoplasms,44753565.34
21,"Symptoms, signs and abnormal clinical and labo...",43113226.25
14,"Endocrine, nutritional and metabolic diseases",37455814.63
9,Diseases of the genitourinary system,37301948.14
11,Diseases of the nervous system,33305126.91
17,"Injury, poisoning and certain other consequenc...",30787867.40


In [105]:
episodes_to_keep = ['Diseases of the musculoskeletal system and connective tissue', 'Neoplasms', 'Diseases of the digestive system', 
                    'Diseases of the circulatory system', 'Diseases of the circulatory system', 'Diseases of the genitourinary system',
                   'Diseases of the nervous system', 'Mental, Behavioral and Neurodevelopmental disorders']
med1 = med
med1['ICD10_CHAPTER'] = med['ICD10_CHAPTER'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [106]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='ICD10_CHAPTER',
    aggfunc='sum',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
MEMBER_ID,,,,,,,
mma-0001ed26305b48f99696f11dfcea17b7,1497.76,0.00,0.00,19534.07,2246.15,0.00,0.00
mma-00024d2d45224e76915920b058364ff1,0.00,0.00,0.00,0.00,0.00,0.00,0.00
mma-0006f22cc2ed4e6da73eb73991bbbf57,964.32,16152.62,5941.18,861.64,0.00,0.00,3557.37
mma-0007e20948f4487d97f4eb2981613ea1,0.00,0.00,0.00,872.44,93.42,0.00,0.00
mma-0008856832474235be7f58214e0cabc3,0.00,0.00,161.01,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...
mma-ffe7972f15694551af377f4666f57f60,0.00,0.00,0.00,0.00,270.09,0.00,0.00
mma-ffe9040f10b9429eb7ce09ceaaf1e158,608.92,0.00,0.00,0.00,0.00,0.00,0.00
mma-fff2da3c741f4879991e6f65a74c58e9,0.00,0.00,11537.00,869.21,0.00,0.00,0.00


In [107]:
member_merge_cond = merge_member.merge(med_meg, on='MEMBER_ID')
member_merge_cond

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
0,mma-036ffc20ff794595a44950a4729f66d3,LA,Subscriber,Female,62,Baby Boomers,1548.18,6302.57,599.78,1894.85,346.67,22.39,-466.28,345.11,11264.21,14074.46,1549.84,1204.73,349.09,642.44,1.00,0,10,1,6,0,897.20,0.00,0.00,4493.81,0.00,0.00,0.00
1,mma-4c45de73f32445699b80d8d493793dff,MA,Subscriber,Male,57,Generation X,3612.20,24394.26,6790.70,19444.98,15832.78,438.31,2989.48,939.01,982.04,2713.93,2531.62,1592.61,169.61,650.97,1.00,13,4,2,2,0,4326.72,0.00,4601.44,36650.69,0.00,0.00,330.51
2,mma-f8d8a13411e045629b92e106d35b3cd8,NJ,Spouse,Male,42,Millenials,1302.25,1166.67,767.91,389.53,-912.72,-70.09,-313.69,522.14,0.00,0.00,48.51,-473.63,-90.71,-142.09,1.00,0,6,1,4,0,0.00,0.00,0.00,1048.98,0.00,0.00,0.00
3,mma-ca2e959e4c094ddab3a345d0d94d2f3e,GA,Subscriber,Female,65,Baby Boomers,3928.97,241.56,525.98,861.81,-3067.16,-78.07,-891.70,1929.71,0.00,0.00,0.00,-1929.71,-100.00,-578.91,1.00,0,4,0,1,0,122.01,0.00,447.47,172.32,0.00,0.00,0.00
4,mma-ed99a4f01306482caec74ec3d315501c,IL,Spouse,Female,38,Millenials,24820.37,288.48,12854.43,1513.65,-23306.72,-93.90,-5735.42,0.00,1392.91,233.62,1520.28,1520.28,0.00,340.15,1.00,0,7,0,4,0,514.00,0.00,0.00,1194.08,277.35,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18406,mma-38955c494c7c4c888d0f65459d8959e3,CO,Subscriber,Female,47,Generation X,1037.07,660.84,651.72,982.45,-54.62,-5.27,-17.30,1603.44,2200.50,1308.08,2179.17,575.73,35.91,83.48,1.00,0,4,0,5,0,0.00,79.30,5.45,0.00,0.00,0.00,0.00
18407,mma-46d15af9af544cd6b09133c45ec04642,WI,Subscriber,Male,47,Generation X,502.23,2096.12,1696.09,73865.80,73363.57,14607.56,21969.07,366.80,1509.65,930.84,22.05,-344.75,-93.99,-161.31,1.00,10,0,0,4,0,0.00,0.00,0.00,64764.14,0.00,0.00,0.00
18408,mma-c0b85487143b49dcbd14a6c3794ba398,DE,Dependent,Female,23,Generation Z,6949.24,2078.82,485.71,0.00,-6949.24,-100.00,-2244.08,0.00,0.00,550.18,846.40,846.40,0.00,308.94,1.00,6,12,0,3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
18409,mma-565511f8a56541379d9fd2f16e419df3,IL,Subscriber,Female,64,Baby Boomers,1326.52,8985.78,765.83,1809.56,483.04,36.41,-677.08,2200.72,2374.92,1961.08,630.36,-1570.36,-71.36,-512.49,1.00,16,13,0,0,0,828.98,123.19,0.00,1383.23,1404.86,0.00,563.15


In [108]:
member_merge_cond.describe()

C:\Users\U1229773\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC,Diseases of the circulatory system,Diseases of the digestive system,Diseases of the genitourinary system,Diseases of the musculoskeletal system and connective tissue,Diseases of the nervous system,"Mental, Behavioral and Neurodevelopmental disorders",Neoplasms
count,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,16565.00,18411.00,18409.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00,18411.00
mean,45.54,7118.14,7519.20,8083.70,8550.79,1432.65,NaN,486.25,17540.63,20948.94,23876.80,24397.67,6857.04,-inf,2349.90,0.98,8.47,12.44,1.48,3.09,3.92,2424.98,2341.90,1889.60,4005.99,1699.93,1300.37,2321.11
std,15.29,25728.77,32617.62,37615.66,31401.11,35658.17,NaN,11807.54,148245.90,194904.23,271350.00,232796.56,224183.15,NaN,81431.45,0.14,21.54,12.79,3.15,2.92,19.56,21146.48,13054.56,25905.97,20636.98,20162.14,13086.16,21921.33
min,4.00,-55270.45,-141763.50,-66332.73,-45832.90,-925844.38,-inf,-279383.92,-50622.43,-11794.24,-6618.00,-21042.98,-8150200.02,-inf,-2569101.15,0.00,0.00,0.00,0.00,0.00,0.00,-4539.46,-34848.03,-1976.80,-16530.98,-2549.35,-195.35,-11237.23
25%,37.00,447.81,462.26,463.93,481.11,-1297.07,-67.97,-574.25,18.53,22.97,5.05,2.90,-580.03,-85.75,-203.53,1.00,0.00,5.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,48.00,1297.53,1376.48,1479.99,1545.16,65.95,7.07,25.30,390.84,408.41,387.20,358.92,0.00,-12.03,-0.77,1.00,0.00,9.00,0.00,3.00,0.00,0.00,0.00,0.00,151.71,0.00,0.00,0.00
75%,57.00,4365.84,4565.94,5031.29,5340.90,2089.43,305.86,869.15,2657.78,2881.07,3097.57,3207.46,575.28,62.60,208.24,1.00,9.00,16.00,2.00,4.00,0.00,336.36,312.61,309.26,1390.94,224.10,213.84,71.38
max,94.00,929755.23,1852142.12,3292397.01,989586.75,981789.36,inf,344595.35,9142459.77,10965077.04,25228207.34,19336560.40,19214650.23,1080133364827837824.00,7440455.72,1.00,527.00,246.00,117.00,71.00,1055.00,1847547.90,603919.46,2725583.06,1497293.30,1233568.23,1104950.23,1279992.94


In [109]:
drugs = rx.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
233,CNS Agents,Anti-Rheumatics/Immune Modulators,Humira (2 Pen),139049491.90
231,CNS Agents,Anti-Rheumatics/Immune Modulators,Enbrel SureClick,50532384.50
2711,Topical Agents,Psoriasis Agents,Taltz,45694352.85
2710,Topical Agents,Psoriasis Agents,Stelara,41496017.80
2707,Topical Agents,Psoriasis Agents,Skyrizi Pen,40127120.12
...,...,...,...,...
1690,Lipid Agents,Niacin Agents,Niacin (Antihyperlipidemic),-853.06
715,CNS Agents,Weight Loss Agents,Saxenda,-1234.35
2040,Nutrional,Prenatal Vitamins,PreGenna,-1483.64
1784,Medical Supplies,Medical Supplies,Contour Next EZ,-45210.55


In [110]:
member_merge_cond.to_csv('no-glp1-members.csv', index=False)

In [59]:
controlled_members = pd.read_csv('matched_pairs_controlled_members.csv')
controlled_members = controlled_members['MEMBER_ID'].unique()

In [61]:
rx_controlled = rx[rx['MEMBER_ID'].isin(controlled_members)]
rx_controlled

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
226,1121944.00,mma-b895a2a37e9548189a8a13297490793f,2021-01-02,2020-12-31,hydroCHLOROthiazide,hydroCHLOROthiazide,0.00,90.00,Cardiac,Diuretics,2021,2020
227,1121944.00,mma-b895a2a37e9548189a8a13297490793f,2021-01-02,2020-12-31,hydroCHLOROthiazide,hydroCHLOROthiazide,0.00,90.00,Cardiac,Diuretics,2021,2020
228,1121944.00,mma-b895a2a37e9548189a8a13297490793f,2021-01-02,2020-12-31,hydroCHLOROthiazide,hydroCHLOROthiazide,0.00,90.00,Cardiac,Diuretics,2021,2020
229,1121944.00,mma-b895a2a37e9548189a8a13297490793f,2021-01-02,2020-12-31,hydroCHLOROthiazide,hydroCHLOROthiazide,0.00,90.00,Cardiac,Diuretics,2021,2020
230,1121944.00,mma-b895a2a37e9548189a8a13297490793f,2021-01-02,2020-12-31,hydroCHLOROthiazide,hydroCHLOROthiazide,0.00,90.00,Cardiac,Diuretics,2021,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
13480618,4936948.00,mma-12bc4c259f804f969c54585de73aa372,2024-12-31,2024-12-14,Cephalexin,Cephalexin,2.51,15.00,Antibiotics,Cephalosporins,2024,2024
13480619,6350322.00,mma-12bc4c259f804f969c54585de73aa372,2024-12-31,2024-12-30,Sertraline HCl,Sertraline HCl,2.18,23.00,CNS Agents,Antidepressants - SSRI,2024,2024
13480620,6350322.00,mma-12bc4c259f804f969c54585de73aa372,2024-12-31,2024-12-30,Sertraline HCl,Sertraline HCl,2.18,23.00,CNS Agents,Antidepressants - SSRI,2024,2024
13480621,6350322.00,mma-12bc4c259f804f969c54585de73aa372,2024-12-31,2024-12-30,Sertraline HCl,Sertraline HCl,2.18,23.00,CNS Agents,Antidepressants - SSRI,2024,2024


In [62]:
med_controlled = med[med['MEMBER_ID'].isin(controlled_members)]
med_controlled

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,PROCEDURE_DESC,ARTTOS_V2_L1,ARTTOS_V2_L3,DX_IS_CHRONIC,ICD10_CHAPTER,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
30,15042239.00,mma-6cb6ef7f2a2048c7a5541746d50ff404,2021-01-01,2021,2020-12-21,2020,87.54,0.00,0.00,"Malignant neoplasm of colon, unspecified",CT of thorax with contrast (CPT 71260),Professional,CT/MRI/PET - CT Scan (P57a),1.00,Neoplasms,Malignant neoplasms of digestive organs,Malignant neoplasm of colon,"Neoplasm, Malignant: Colon and Rectum",0,0,0,0,0,0,0
46,4465440.00,mma-9199a25a48824455bf582697a6eaf021,2021-01-01,2021,2020-12-21,2020,9.32,0.00,0.00,Encounter for general adult medical examinatio...,"Blood count; complete (CBC), automated (Hgb, H...",Professional,Office - Independent Lab (P63c),0.00,OTHER,Persons encountering health services for exami...,Encounter for general examination without comp...,Encounter for Preventive Health Services,0,0,0,0,0,0,0
125,12728927.00,mma-a127fdea091341c9a669d6439084f16b,2021-01-01,2021,2020-02-21,2020,-6.18,0.00,0.00,"Unspecified injury of unspecified ankle, initi...","Complete X-ray of ankle, anteroposterior, late...",Professional,General - Diagnostic (P56b),0.00,OTHER,Injuries to the ankle and foot,Other and unspecified injuries of ankle and foot,"Injury, Open Wound, or Blunt Trauma: Lower Ext...",0,0,0,0,0,0,0
126,51038.00,mma-a127fdea091341c9a669d6439084f16b,2021-01-01,2021,2020-02-21,2020,21.98,0.00,0.00,"Unspecified injury of unspecified ankle, initi...","Complete X-ray of ankle, anteroposterior, late...",Professional,General - Diagnostic (P56b),0.00,OTHER,Injuries to the ankle and foot,Other and unspecified injuries of ankle and foot,"Injury, Open Wound, or Blunt Trauma: Lower Ext...",0,0,0,0,0,0,0
136,6673077.00,mma-029016ca28ef4fb1a18d884c1021553c,2021-01-01,2021,2020-12-03,2020,0.00,0.00,0.00,"Body mass index [BMI] 35.0-35.9, adult","Body Mass Index (BMI), documented (PV) (CPT 30...",Other,Documentation/Unclassified (P99g),1.00,OTHER,Body mass index [BMI],Body mass index [BMI],Overweight and Obesity,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880090,2377938.00,mma-d09bb724fd5a4ae39046642c60dde5e9,2024-12-31,2024,2024-12-10,2024,0.00,1.00,0.00,Malignant neoplasm of upper-outer quadrant of ...,Office or other outpatient visit for the evalu...,Professional,Specialist (P32d),1.00,Neoplasms,Malignant neoplasms of breast,Malignant neoplasm of breast,"Neoplasm, Malignant: Breast, Female",0,0,1,0,0,0,0
3880095,8699912.00,mma-d09bb724fd5a4ae39046642c60dde5e9,2024-12-31,2024,2024-12-19,2024,-0.01,0.00,0.00,Malignant neoplasm of upper-outer quadrant of ...,"Blood count; reticulocytes, automated, includi...",Professional,Office - General (P63a),1.00,Neoplasms,Malignant neoplasms of breast,Malignant neoplasm of breast,"Neoplasm, Malignant: Breast, Female",0,0,0,0,0,0,0
3880101,6526312.00,mma-d09bb724fd5a4ae39046642c60dde5e9,2024-12-31,2024,2024-12-19,2024,12.38,0.00,0.00,Malignant neoplasm of upper-outer quadrant of ...,"Blood test, comprehensive group of blood chemi...",Professional,Office - General (P63a),1.00,Neoplasms,Malignant neoplasms of breast,Malignant neoplasm of breast,"Neoplasm, Malignant: Breast, Female",0,0,0,0,0,0,0
3880104,8699912.00,mma-d09bb724fd5a4ae39046642c60dde5e9,2024-12-31,2024,2024-12-19,2024,0.00,0.00,0.00,Malignant neoplasm of upper-outer quadrant of ...,Insertion of needle into vein for collection o...,Professional,Office - Venipuncture (P63b),1.00,Neoplasms,Malignant neoplasms of breast,Malignant neoplasm of breast,"Neoplasm, Malignant: Breast, Female",0,0,0,0,0,0,0


In [63]:
member_controlled = member[member['MEMBER_ID'].isin(controlled_members)]
member_controlled

,index,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
9,41,mma-5c6f80aabb5140e3ad2916c50cfd7cd6,0.00,NC,Subscriber,Female,38,Millenials
14,53,mma-769f8724e508428a8a7ad17105b5a8cc,0.00,FL,Subscriber,Female,63,Baby Boomers
15,54,mma-df5b9938e46d4f05b608e89b84d45047,0.00,KY,Subscriber,Male,37,Millenials
17,56,mma-55bc2d90ae2a4ca19a6c821f16a0b26c,0.00,MN,Subscriber,Female,43,Millenials
19,60,mma-007ed462b23b4c82a8153a7cc91d2367,0.00,MN,Subscriber,Female,41,Millenials
...,...,...,...,...,...,...,...,...
19876,151735,mma-87d877bea8b84356a6fd0d5350982875,0.00,MS,Spouse,Female,38,Millenials
19955,152164,mma-e0e9cf70a43b4aaba27ddee386f24b3c,0.00,OH,Subscriber,Female,50,Generation X
20048,152954,mma-6d39c88eef4d433a8527f5672da78525,0.00,OH,Spouse,Female,57,Generation X
20127,153679,mma-23e8d30aeef245a489e56fa3f74ba5b3,0.00,KS,Subscriber,Female,61,Baby Boomers


In [64]:
pip install pandas openpyxl

Looking in indexes: https://mgti-dal-so-art.mrshmc.com/artifactory/api/pypi/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [17]:
import openpyxl

In [19]:
rx_controlled.to_excel('rx-controlled-claims.xlsx', index=False, engine='openpyxl')
med_controlled.to_excel('med-controlled-claims.xlsx', index=False, engine='openpyxl')
member_controlled.to_excel('member-controlled-claims.xlsx', index=False, engine='openpyxl')